In [4]:
print("hi")

hi


In [1]:
# imports
import os, sys

# third party imports
import numpy as np
import tensorflow as tf
assert tf.__version__.startswith('2.'), 'This tutorial assumes Tensorflow 2.0+'

# local imports
import voxelmorph as vxm
import neurite as ne
import matplotlib.pyplot as plt
from tensorflow.keras import models, layers, activations, initializers, regularizers, optimizers, losses, callbacks
import numpy as np
import pandas as pd
import cv2 as cv
import glob
import os
from scipy import ndimage
from IPython import display 
import matplotlib.pyplot as plt
from tqdm import tqdm
import nibabel as nib
from ants import from_numpy, resample_image, registration, apply_transforms
import pydicom


In [2]:
Moving_phase = 'A'
fixed_phase  = 'D'
patch_size = 64


train_A = "/media/monib/ext1/work2022/Base_Dataset/vm_data_affine_a2d/input/train/example_A"
train_D = "/media/monib/ext1/work2022/Base_Dataset/vm_data_affine_a2d/input/train/example_B"
val_A = "/media/monib/ext1/work2022/Base_Dataset/vm_data_affine_a2d/input/val/example_A"
val_D = "/media/monib/ext1/work2022/Base_Dataset/vm_data_affine_a2d/input/val/example_B"

train_A_paths = [os.path.join(train_A, folder) for folder in os.listdir(train_A)]
train_D_paths = [os.path.join(train_D, folder) for folder in os.listdir(train_D)]
val_A_paths = [os.path.join(val_A, folder) for folder in os.listdir(val_A)]
val_D_paths = [os.path.join(val_D, folder) for folder in os.listdir(val_D)]


train_A_paths.sort()
train_D_paths.sort()
val_A_paths.sort()
val_D_paths.sort()

#
Train_num  = len(train_A_paths)
Val_num = len(val_A_paths)

In [4]:
def read_dicom_files(dicom_dir):
    try:
        dicom_files = glob.glob(os.path.join(dicom_dir, "*.*"))
        sorted_dicom_files = sorted(dicom_files)
        stacked_dicom = [pydicom.dcmread(dicom_file) for dicom_file in sorted_dicom_files]
        return stacked_dicom
    except IndexError as e:
        print(f"{e}, at path {dicom_dir}")
        return None

def create_3d(dicom_files):
    stacked_dicom = dicom_files  # stack of dicom files in a list

    image_shape = list(stacked_dicom[0].pixel_array.shape)
    image_shape.append(len(stacked_dicom))
    image_3d = np.zeros(image_shape)

    for j in range(len(stacked_dicom)):
        image_3d[:, :, j] = stacked_dicom[j].pixel_array

    return image_3d

def make_patches(image, patch_size= 64):
    stride = int(patch_size)

    image_patches = []
    locations = []
    for i in range(0, image.shape[0], stride):
        for j in range(0, image.shape[1], stride):
            Patch = np.zeros((patch_size,patch_size,192))
            Patch[Patch==0]= -1024/10000
            img_patch = image[i:i + patch_size,j:j + patch_size, :]
 
            Patch[: img_patch.shape[0],:img_patch.shape[1],:] = img_patch

            patch_img = Patch
            image_patches.append(patch_img)
            locations.append((i, j))

    patch_array = np.zeros((len(image_patches),patch_size,patch_size,image.shape[-1]))
    # print("len img patch", len(image_patches))
    
    for idx in range(len(image_patches)):
        patch_array[idx,:,:,:] =image_patches[idx]
    return patch_array, locations

In [7]:
T_moving = np.zeros((512,512,192))
T_fixed  = np.zeros((512,512,192))

# for i in tqdm(range(Train_num)):
#     moving = create_3d(read_dicom_files(train_P_paths[i]))/10000
#     fixed = create_3d(read_dicom_files(train_A_paths[i]))/10000
#     T_moving[...,:moving.shape[-1]] = moving
#     T_fixed[...,:fixed.shape[-1]] = fixed
#
# # print("Training Moving Data info")
# # print(T_moving.shape)
# # print(T_moving.min())
# # print(T_moving.max())
# #
# # print("Training Fixed Data info")
# # print(T_fixed.shape)
# # print(T_fixed.min())
# # print(T_fixed.max())
#
#
# V_moving = np.zeros((512,512,192))
# V_fixed  = np.zeros((512,512,192))
#
# for i in tqdm(range(Val_num)):
#     moving = create_3d(read_dicom_files(val_P_paths[i]))/10000
#     fixed = create_3d(read_dicom_files(val_A_paths[i]))/10000
#     V_moving[...,:moving.shape[-1]] = moving
#     V_fixed[...,:fixed.shape[-1]] = fixed

# print("Val Moving Data info")
# print(V_moving.shape)
# print(V_moving.min())
# print(V_moving.max())
#
# print("Val Fixed Data info")
# print(V_fixed.shape)
# print(V_fixed.min())
# print(V_fixed.max())

patch_save_path = f'./patch_data_{Moving_phase}_{fixed_phase}_{patch_size}'

try:
    os.makedirs(patch_save_path)
except FileExistsError as err:
    print(err)
else:
    print(patch_save_path)

patch_number = int((512/patch_size)*(512/patch_size))

for i in tqdm(range(Train_num)):
    # TODO: create single image and save it as an array of patches
    moving = create_3d(read_dicom_files(train_A_paths[i]))/10000
    fixed = create_3d(read_dicom_files(train_D_paths[i]))/10000
    if moving.shape[-1] < 192:
        
        T_moving[...,:moving.shape[-1]] = moving
        T_fixed[...,:fixed.shape[-1]] = fixed
    else:
        continue
    
        
    moving_patches , _ = make_patches(T_moving,patch_size)
    fixed_patches  , _ = make_patches(T_fixed,patch_size)

    for j in range(patch_number):
        moving = moving_patches[j:j+1]
        fixed = fixed_patches[j:j+1]
        np.savez(f'{patch_save_path}/{i*patch_number+j}.npz',moving=moving,fixed=fixed)

[Errno 17] File exists: './patch_data_A_D_64'


 27%|█████████████████████▋                                                          | 172/634 [1:38:29<4:53:40, 38.14s/it]/home/monib/Downloads/d/envs/vm2/lib/python3.8/site-packages/pydicom/charset.py:746: UserWarning:

Unknown encoding 'ISO IR 149' - using default encoding instead

 62%|█████████████████████████████████████████████████▍                              | 392/634 [3:47:41<2:04:48, 30.94s/it]/home/monib/Downloads/d/envs/vm2/lib/python3.8/site-packages/pydicom/charset.py:754: UserWarning:

Unknown encoding 'ISO_IR 149' - using default encoding instead

100%|██████████████████████████████████████████████████████████████████████████████████| 634/634 [5:30:25<00:00, 31.27s/it]
